In [505]:
import numpy as np
import obspy
import scipy
from obspy import read
import matplotlib.pyplot as plt 
import time

from __future__ import print_function
import keras
import tensorflow as tf
from keras.layers import add, Reshape, Dense,Input, TimeDistributed, Dropout, Activation, LSTM, Conv2D, Bidirectional, BatchNormalization 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping,ReduceLROnPlateau
from keras.regularizers import l1
from keras import backend as K
from keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import os
np.seterr(divide='ignore', invalid='ignore')
import h5py
from obspy.signal.trigger import trigger_onset
np.warnings.filterwarnings('ignore')

## Grillo Data

In [448]:
s1 = read("/Users/borongwang/Desktop/LSE/PC/lse_challenge/data/signal/1512114894_015_P.mseed")
print(len(s1))

3


In [449]:
s1

3 Trace(s) in Stream:
.015..BHX | 2017-12-01T07:54:54.001000Z - 2017-12-01T07:55:57.969000Z | 31.2 Hz, 2000 samples
.015..BHY | 2017-12-01T07:54:54.001000Z - 2017-12-01T07:55:57.969000Z | 31.2 Hz, 2000 samples
.015..BHZ | 2017-12-01T07:54:54.001000Z - 2017-12-01T07:55:57.969000Z | 31.2 Hz, 2000 samples

In [450]:
s1.plot(outfile="signal1.png")

In [451]:
tr_s1 = s1[1]
trace_data = tr_s1.data
print(type(trace_data))
print(trace_data)

<class 'numpy.ndarray'>
[ 0.          0.01803802 -0.04592396 ...,  0.02292396  0.01096198
  0.        ]


In [452]:
len(trace_data)

2000

### Detrend

In [453]:
s = s1.detrend("demean")

In [454]:
s[1].data

array([ 0.00228925,  0.02032727, -0.04363471, ...,  0.02521321,
        0.01325123,  0.00228925])

In [455]:
len(s[1].data)

2000

### Bandpass filter between 1 and 45Hz

In [456]:
s = s.filter("bandpass", freqmin=1, freqmax=45)

In [457]:
s[1].data

array([ 0.00175945,  0.0146993 , -0.0424117 , ...,  0.00657167,
       -0.00272553, -0.0068781 ])

In [458]:
len(s[1].data)

2000

### Resampled at 100 HZ

In [459]:
s = s.resample(100)

In [460]:
s[1].data

array([ 0.00283503,  0.0039782 ,  0.00282766, ..., -0.003435  ,
       -0.00170013,  0.00060302])

In [461]:
len(s[1].data)

6400

### Normalize

In [462]:
s = s.normalize()

In [463]:
s[1].data

array([ 0.00117299,  0.00164597,  0.00116994, ..., -0.00142122,
       -0.00070343,  0.0002495 ])

In [464]:
len(s[1].data)

6400

In [465]:
s.plot(outfile="tf.png")

### STFT

In [473]:
from scipy import signal

In [474]:
f, t, Zxx = signal.stft(s, fs=100,nperseg=80)
print(Zxx.shape)

(3, 41, 161)


In [475]:
#Zxx = np.abs(Zxx)
#Zxx = Zxx.T
Zxx.shape

(3, 41, 161)

In [476]:
max(f),min(f)

(50.0, 0.0)

In [477]:
max(t)-min(t)

64.0

In [478]:
fig, ax = plt.subplots()
#f, t, Zxx = signal.stft(sig, fs, window='hann', nperseg=1024)
ind=np.argsort(f) # returns the indices of the sorted f vector
f=f[ind]
Zxx=Zxx[:,ind,:]
plt.pcolormesh(t, f, np.abs(Zxx[0,:,:]*np.conj(Zxx[0,:,:])), shading='gouraud')
plt.plot(outfile="stft.png")

[]

## Code

In [509]:
noise_filename = pd.read_csv('noise.csv', header = None)
noise_stream = []
for i in noise_filename[0]:
    temp = obspy.read("data/noise/" + i)
    noise_stream.append(temp)
    
signal_filename = pd.read_csv('signal.csv', header = None)
signal_stream = []
for i in signal_filename[0]:
    temp = obspy.read("data/signal/" + i)
    signal_stream.append(temp)

In [510]:
def slice_stream(st):
    start_time = st[0].stats.starttime
    end_time = st[0].stats.endtime
    gap = (end_time - start_time)/4
    st = st.slice(start_time + gap, end_time - gap)
    return st

In [511]:
noise_stream = list(map(slice_stream,noise_stream))
signal_stream = list(map(slice_stream,signal_stream))

In [512]:
noise_stream = [n.filter("bandpass", freqmin=1, freqmax=45) for n in noise_stream]
noise_stream = [n.resample(100) for n in noise_stream]
noise_stream = [n.detrend("demean") for n in noise_stream]
noise_stream = [n.normalize() for n in noise_stream]

In [513]:
signal_stream = [n.filter("bandpass", freqmin=1, freqmax=45) for n in signal_stream]
signal_stream = [n.resample(100) for n in signal_stream]
signal_stream = [n.detrend("demean") for n in signal_stream]
signal_stream = [n.normalize() for n in signal_stream]

In [514]:
len(noise_stream), len(signal_stream)

(17850, 1223)

In [515]:
signal_stream[0].plot(outfile="signal11.png")

In [516]:
f, t, Zxx = signal.stft(signal_stream[0], fs = 100, nperseg=50)

In [517]:
Zxx = np.abs(Zxx).T
Zxx.shape

(129, 26, 3)

In [518]:
noise_length = len(noise_stream)
noise_y = np.zeros(shape = (noise_length, 33, 1))

signal_length = len(signal_stream)
signal_y_truth = np.append(np.zeros(shape=(16,1)), np.ones(shape=(17,1)),axis=0)
signal_y = np.zeros(shape = (signal_length, 33, 1))
for i in range(signal_length):
    signal_y[i] = signal_y_truth

In [519]:
f_signal = []
t_signal = []
Zxx_signal = np.zeros(shape = (signal_length, 129, 26, 3))

for i in range(signal_length):
    f, t, Zxx = signal.stft(signal_stream[i], fs = 100, nperseg=50)
    f_signal.append(f)
    t_signal.append(t)
    Zxx_signal[i] = np.abs(Zxx).T

f_noise = []
t_noise = []
Zxx_noise = np.zeros(shape = (noise_length, 129, 26, 3))

for i in range(noise_length):
    f, t, Zxx = signal.stft(noise_stream[i], fs = 100, nperseg=50)
    f_noise.append(f)
    t_noise.append(t)
    Zxx_noise[i] = np.abs(Zxx).T

In [520]:
y = np.append(noise_y, signal_y, axis = 0)
X_Zxx = np.append(Zxx_noise, Zxx_signal, axis = 0)

In [521]:
print(y.shape)
print(X_Zxx.shape)

(19073, 33, 1)
(19073, 129, 26, 3)


## Model CRED

In [522]:
def lr_schedule(epoch):
    """
    Learning rate is scheduled to be reduced after 40, 60, 80, 90 epochs.
    """
    lr = 1e-3
    if epoch > 60:
        lr *= 0.5e-3
    elif epoch > 40:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [523]:
def block_CNN(filters, ker, inpC): 
    """
    Returns CNN residual blocks
    """
    layer_1 = BatchNormalization()(inpC) 
    act_1 = Activation('relu')(layer_1) 

    conv_1 = Conv2D(filters, (ker-2, ker-2), padding = 'same')(act_1) 
    
    layer_2 = BatchNormalization()(conv_1) 
    act_2 = Activation('relu')(layer_2) 
  
    conv_2 = Conv2D(filters, (ker-2, ker-2), padding = 'same')(act_2) 
    return(conv_2) 

In [524]:
def block_BiLSTM(inpR, filters, rnn_depth):
    """
    Returns LSTM residual blocks
    """
    x = inpR
    for i in range(rnn_depth):
        x_rnn = Bidirectional(LSTM(filters, return_sequences=True))(x)
        x_rnn = Dropout(0.7)(x_rnn)
        if i > 0 :
            x = add([x, x_rnn])
        else:
            x = x_rnn      
    return x
     

In [525]:
def model_cred(shape, filters):
    
    inp = Input(shape=shape, name='input')

    conv2D_2 = Conv2D(filters[0], (9,9), strides = (2,2), padding = 'same', activation = 'relu')(inp) 
    res_conv_2 = keras.layers.add([block_CNN(filters[0], 9, conv2D_2), conv2D_2]) 

    conv2D_3 = Conv2D(filters[1], (5,5), strides = (2,2), padding = 'same', activation = 'relu')(res_conv_2) 
    res_conv_3 = keras.layers.add([block_CNN(filters[1], 5, conv2D_3),conv2D_3]) 
    
    shape = K.int_shape(res_conv_3)   
    reshaped = Reshape((shape[1], shape[2]*shape[3]))(res_conv_3)
    
    res_BIlstm = block_BiLSTM(reshaped, filters = filters[3], rnn_depth = 2)
 
    UNIlstm = LSTM(filters[3], return_sequences=True)(res_BIlstm)
    UNIlstm = Dropout(0.8)(UNIlstm)  
    UNIlstm = BatchNormalization()(UNIlstm)
   
    dense_2 = TimeDistributed(Dense(filters[3], kernel_regularizer=l1(0.01), activation='relu'))(UNIlstm)
    dense_2 = BatchNormalization()(dense_2)
    dense_2 = Dropout(0.8)(dense_2)
    
    dense_3 = TimeDistributed(Dense(1, kernel_regularizer=l1(0.01), activation='sigmoid'))(dense_2)

    out_model = Model(inputs=inp, outputs=dense_3)
    return out_model  

In [526]:
early_stopping_monitor = EarlyStopping(patience=5)
    
lr_scheduler = LearningRateScheduler(lr_schedule)
    
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5-2,
                                   min_lr=0.5e-7)

In [527]:
model = model_cred((129, 26, 3), filters = [8, 16, 32, 64, 128, 256])
model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(lr=lr_schedule(0)),
                  metrics=['binary_accuracy'])

model.summary()

Learning rate:  0.001
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 129, 26, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_48 (Conv2D)             (None, 65, 13, 8)    1952        ['input[0][0]']                  
                                                                                                  
 batch_normalization_48 (BatchN  (None, 65, 13, 8)   32          ['conv2d_48[0][0]']              
 ormalization)                                                                                    
                                                                      

In [528]:
seed_value = 15
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

In [529]:
X, X_val, y, y_val = train_test_split(X_Zxx, y, test_size=0.3, random_state=15,shuffle=True)

In [530]:
checkpointer = ModelCheckpoint(filepath='model_best.hdf5',
                                       monitor='val_loss', verbose=0, mode='auto', save_best_only=True)
start_time = time.time()  
history = model.fit(
    X,
    y,
    epochs=200,
    batch_size=500,
    verbose=0,
    validation_data = (X_val, y_val),
    max_queue_size=5,
    callbacks = [checkpointer, lr_reducer, lr_scheduler, early_stopping_monitor] )
end_time = time.time()

Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001


In [532]:
history.history

{'loss': [5.081862926483154,
  3.7337281703948975,
  2.714608907699585,
  1.8995134830474854,
  1.245247483253479,
  0.7525756359100342,
  0.4200800955295563,
  0.2511064410209656,
  0.2103590965270996,
  0.20754188299179077,
  0.24780790507793427,
  0.21268294751644135,
  0.1976938247680664,
  0.19360005855560303,
  0.19240334630012512,
  0.19013643264770508,
  0.18881116807460785],
 'binary_accuracy': [0.554474413394928,
  0.7265918254852295,
  0.9354945421218872,
  0.987309992313385,
  0.9944800138473511,
  0.9965137243270874,
  0.9978097081184387,
  0.9984724521636963,
  0.9982069134712219,
  0.9970561861991882,
  0.9925984144210815,
  0.9975214600563049,
  0.9977030158042908,
  0.9978846311569214,
  0.9979368448257446,
  0.9979572296142578,
  0.9980548620223999],
 'val_loss': [5.620278835296631,
  4.747651100158691,
  4.513829708099365,
  3.864873170852661,
  3.1404354572296143,
  2.043123722076416,
  1.282875657081604,
  1.032277226448059,
  1.1293171644210815,
  1.40609622001647

In [ ]:
model.load_weights('model_best.hdf5')
mse = model.evaluate(Xt_1, yt_1, verbose=0)
mse_list.append(mse)     